In [1]:
# Importing modules
import numpy as np
import pandas as pd
import numba
import pyarrow.dataset as ds

In [2]:
# Pandas display options (personal preference)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [3]:
# Importing the G09 parquet file
dataset = ds.dataset("../../data/WD03/WD03_2p4.parquet", format="parquet")
table = dataset.to_table()

# Setting uberID as index
df = table.to_pandas().set_index('uberID')
df.index=df.index.astype('int64')

del dataset,table

In [4]:
# Getting rid of weird spaces in the class column
df=df.replace({'class': {'ambiguous ': 'ambiguous',
                         'star      ': 'star', 
                         'galaxy    ': 'galaxy', 
                         'artefact  ':'artefact'}})

In [5]:
# Making a number of cuts: magnitude, mask, starmask and removing artefacts
df=df[(df['mag_Zt']<22) & 
      (df['mask']==0) & 
      (df['starmask']==0) & 
      (df['class']!='artefact')&
      (df['duplicate']==0)
]

In [6]:
# Getting a 1/10 subset of the data (for the purpose of this tutorial)
# df=df[::10]

In [7]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
53741013700771,waves_postprocessed_53.9_-26.6.rds,5374.0,458,10136.5,770.5,6705.157716,53.606390,-27.118851,53.606355,-27.118805,53.606483,53.606390,-27.118792,-27.118875,53.606473,-27.118865,53.606413,-27.118868,53.606448,-27.118889,53.606438,-27.118879,53.606448,-27.118889,53.606391,-27.118851,53.606355,-27.118805,0.006383,0.025301,-0.072686,0.643380,0.642938,-0.072271,-0.086977,16.063300,0.014180,0.836855,0.766050,0.745892,0.857463,0.680968,0.622480,2.260227,6.599540,1445,0.950459,21.008265,458,1,16.933797,16.012305,17.330255,16.841586,16.677668,16.948896,2.821301e-04,2.563763e-07,2.649473e-04,2.407621e-07,0.000885,1.524212e-07,0.000844,1.452980e-07,0.001265,4.687419e-07,0.001222,4.527694e-07,0.001394,4.071650e-07,0.001357,3.962806e-07,0.001429,1.159134e-06,0.001400,1.135873e-06,0.001441,2.266972e-06,0.001418,2.231345e-06,0.001245,1.480766e-06,0.001231,1.463917e-06,0.001136,4.675483e-06,0.001128,4.641061e-06,0.000787,3.212514e-06,0.000784,3.197075e-06,0.000342,1.911904e-06,0.000341,1.906886e-06,0.000181,1.502790e-06,0.000181,1.500050e-06,2.792261e-04,1.911977e-07,0.000877,1.102875e-07,0.001253,2.620950e-07,0.001381,2.948605e-07,0.001429,1.159134e-06,0.001441,2.266972e-06,0.001236,8.145253e-07,0.001136,4.675483e-06,0.000780,1.754387e-06,0.000289,4.053011e-07,1.435894e-04,4.360427e-07,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,0.0,16.012305,0
53740752801525,waves_postprocessed_53.9_-26.6.rds,5374.0,472,7527.5,1524.5,4856.520256,53.850608,-27.056279,53.850641,-27.056273,53.850676,53.850676,-27.056260,-27.056260,53.850653,-27.056261,53.850614,-27.056299,53.850641,-27.056273,53.850618,-27.056275,53.850641,-27.056273,53.850608,-27.056279,53.850641,-27.056273,-0.001477,0.024759,-0.072686,0.643380,0.642938,-0.072271,-0.086977,16.059592,0.015323,0.923591,0.775714,0.787018,0.800750,0.655798,0.654888,2.455160,6.255933,1277,0.934759,18.303824,472,3,17.014767,16.008782,17.516480,16.927034,16.710907,16.725505,1.621672e-04,2.615204e-07,1.515208e-04,2.443515e-07,0.000715,2.092271e-07,0.000679,1.986807e-07,0.001202,2.557735e-07,0.001158,2.463682e-07,0.001390,6.542980e-07,0.001350,6.354171e-07,0.001434,1.212677e-06,0.001403,1.186400e-06,0.001534,2.323633e-06,0.001508,2.284195e-06,0.001389,1.475077e-06,0.001371,1.456948e-06,0.001323,5.080794e-06,0.001313,5.040383e-06,0.000930,1.368180e-06,0.000925,1.361075e-06,0.000428,1.6375

In [8]:
# Reading the u and no u labels from notebooks 1 and 2, and creating new dataframes
df_u=pd.read_csv('../../data/WD03/WD03_u.csv').set_index('uberID')
df_no_u=pd.read_csv('../../data/WD03/WD03_no_u.csv').set_index('uberID')

In [9]:
# Creating new columns in the main dataframe with u and no u labels
df['cluster_label']=np.zeros(len(df))
df['cluster_label_no_u']=np.zeros(len(df))

df['label_type']=np.zeros(len(df))
df['label_type']='missing_band'

In [10]:
# Creating umap columns in the main dataframe with u and no u labels
for i in range(10):
    df[f'umap{i+1}_no_u'] = np.zeros(len(df))
    df[f'umap{i+1}_u'] = np.zeros(len(df))

In [11]:
# Filling the new columns with u and no u labels
df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)

/tmp/ipykernel_137417/2692133000.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'star' 'star' ... 'galaxy' 'galaxy' 'galaxy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_u.index,'cluster_label']=df_u['cluster_label'].apply(str)
/tmp/ipykernel_137417/2692133000.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['star' 'star' 'star' ... 'galaxy' 'galaxy' 'galaxy']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df_no_u.index,'cluster_label_no_u']=df_no_u['cluster_label'].apply(str)


In [12]:
for i in range(10):
    df.loc[df_u.index,f'umap{i+1}_u']=df_u[f'umap{i+1}']
    df.loc[df_no_u.index,f'umap{i+1}_no_u']=df_no_u[f'umap{i+1}']

In [13]:
labels=['star','galaxy']

In [14]:
# Creating a 'final label' column full of 1's to combine the two labels
df['final_label']=np.ones(len(df))


In [15]:
# Creating umap columns with 1's
for i in range(10):
    df[f'umap{i+1}'] = np.ones(len(df))


In [16]:
# First applying the no u labels to the final label
for label in labels:
    df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label
    df.loc[df[df['cluster_label_no_u']==label].index,'label_type']='no_u'

    # First putting umap columns with no u labels
    for i in range(10):
        df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label_no_u']==label].index, f'umap{i+1}_no_u']

/tmp/ipykernel_137417/2715295979.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'star' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df[df['cluster_label_no_u']==label].index,'final_label']=label


In [17]:
# Then applying the u labels to the final label, overwriting any existing no u labels
for label in labels:
    df.loc[df[df['cluster_label']==label].index,'final_label']=label
    df.loc[df[df['cluster_label']==label].index,'label_type']='u'

    for i in range(10):
        df.loc[df[df['cluster_label']==label].index, f'umap{i+1}'] = df.loc[df[df['cluster_label']==label].index, f'umap{i+1}_u']

In [18]:
# Convering final label to string
df['final_label']=df['final_label'].apply(str)

In [19]:
# Creating a new column to indicate whether a source has any missing bands.
# If final label still == 1, that indicates a label hasn't been added, so it has a missing band
df['missing_bands']=True
df.loc[df[df['final_label']!='1.0'].index,'missing_bands']=False

In [20]:
# For the sources with missing labels we then overwrite with class
df.loc[df[df['final_label']=='1.0'].index,'final_label'] = df[df['final_label']=='1.0']['class']

In [21]:
# For sources with missing labels we put nan in the umap columns
for i in range(10):
    df.loc[df[df['missing_bands']==True].index, f'umap{i+1}'] = np.nan

In [22]:
# Sources with missing bands are only 1.5% of the sample
len(df[df['missing_bands']==True])/len(df)

0.007639998361394454

In [23]:
df

,FrameName,FrameID,segID,xmax,ymax,censep,RAcen,Deccen,RAmax,Decmax,RAGAIA_r,RAGAIA,DecGAIA,DecGAIA_r,RAGAIA_r_cen,DecGAIA_r_cen,RAcen_gt,Deccen_gt,RAmax_gt,Decmax_gt,RAcen_rt,Deccen_rt,RAmax_rt,Decmax_rt,RAcen_Zt,Deccen_Zt,RAmax_Zt,Decmax_Zt,sky_mean,skyRMS_mean,log10seeing,log10seeing_r,log10seeing_i,log10seeing_Z,log10seeing_Y,mag,EBV,R50,R50_gt,R50_rt,R50_Yt,R50_Jt,R50_Ht,R90,R100,N100,axrat,ang,groupID,Ngroup,mag_app_Zt,mag_Zt,mag_app_gt,mag_app_rt,mag_app_it,mag_app_Yt,flux_ut,flux_err_ut,flux_ut_uncorrected,flux_err_ut_uncorrected,flux_gt,flux_err_gt,flux_gt_uncorrected,flux_err_gt_uncorrected,flux_rt,flux_err_rt,flux_rt_uncorrected,flux_err_rt_uncorrected,flux_it,flux_err_it,flux_it_uncorrected,flux_err_it_uncorrected,flux_Zt,flux_err_Zt,flux_Zt_uncorrected,flux_err_Zt_uncorrected,flux_Yt,flux_err_Yt,flux_Yt_uncorrected,flux_err_Yt_uncorrected,flux_Jt,flux_err_Jt,flux_Jt_uncorrected,flux_err_Jt_uncorrected,flux_Ht,flux_err_Ht,flux_Ht_uncorrected,flux_err_Ht_uncorrected,flux_Kt,flux_err_Kt,flux_Kt_uncorrected,flux_err_Kt_uncorrected,flux_W1t,flux_err_W1t,flux_W1t_uncorrected,flux_err_W1t_uncorrected,flux_W2t,flux_err_W2t,flux_W2t_uncorrected,flux_err_W2t_uncorrected,flux_uc,flux_err_uc,flux_gc,flux_err_gc,flux_rc,flux_err_rc,flux_ic,flux_err_ic,flux_Zc,flux_err_Zc,flux_Yc,flux_err_Yc,flux_Jc,flux_err_Jc,flux_Hc,flux_err_Hc,flux_Kc,flux_err_Kc,flux_W1c,flux_err_W1c,flux_W2c,flux_err_W2c,mask,starmask,starscol,starssize,class,noOPT_r,noOPT_i,noIR_Z,noIR_Y,duplicate,mag_Zc,mag_Zt_fake,cluster_label,cluster_label_no_u,label_type,umap1_no_u,umap1_u,umap2_no_u,umap2_u,umap3_no_u,umap3_u,umap4_no_u,umap4_u,umap5_no_u,umap5_u,umap6_no_u,umap6_u,umap7_no_u,umap7_u,umap8_no_u,umap8_u,umap9_no_u,umap9_u,umap10_no_u,umap10_u,final_label,umap1,umap2,umap3,umap4,umap5,umap6,umap7,umap8,umap9,umap10,missing_bands
uberID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
53741013700771,waves_postprocessed_53.9_-26.6.rds,5374.0,458,10136.5,770.5,6705.157716,53.606390,-27.118851,53.606355,-27.118805,53.606483,53.606390,-27.118792,-27.118875,53.606473,-27.118865,53.606413,-27.118868,53.606448,-27.118889,53.606438,-27.118879,53.606448,-27.118889,53.606391,-27.118851,53.606355,-27.118805,0.006383,0.025301,-0.072686,0.643380,0.642938,-0.072271,-0.086977,16.063300,0.014180,0.836855,0.766050,0.745892,0.857463,0.680968,0.622480,2.260227,6.599540,1445,0.950459,21.008265,458,1,16.933797,16.012305,17.330255,16.841586,16.677668,16.948896,2.821301e-04,2.563763e-07,2.649473e-04,2.407621e-07,0.000885,1.524212e-07,0.000844,1.452980e-07,0.001265,4.687419e-07,0.001222,4.527694e-07,0.001394,4.071650e-07,0.001357,3.962806e-07,0.001429,1.159134e-06,0.001400,1.135873e-06,0.001441,2.266972e-06,0.001418,2.231345e-06,0.001245,1.480766e-06,0.001231,1.463917e-06,0.001136,4.675483e-06,0.001128,4.641061e-06,0.000787,3.212514e-06,0.000784,3.197075e-06,0.000342,1.911904e-06,0.000341,1.906886e-06,0.000181,1.502790e-06,0.000181,1.500050e-06,2.792261e-04,1.911977e-07,0.000877,1.102875e-07,0.001253,2.620950e-07,0.001381,2.948605e-07,0.001429,1.159134e-06,0.001441,2.266972e-06,0.001236,8.145253e-07,0.001136,4.675483e-06,0.000780,1.754387e-06,0.000289,4.053011e-07,1.435894e-04,4.360427e-07,0.0,0.0,3.0,3.0,star,0.0,0.0,0.0,0.0,0.0,16.012305,0,star,star,u,9.405688,8.461037,5.142072,5.215351,0.704574,-0.208818,5.871566,6.758351,0.973216,1.406018,4.079058,4.857153,5.902344,6.534786,5.414258,5.635551,5.284591,5.968680,5.691504,6.161341,star,8.461037,5.215351,-0.208818,6.758351,1.406018,4.857153,6.534786,5.635551,5.968680,6.161341,False
53740752801525,waves_postprocessed_53.9_-26.6.rds,5374.0,472,7527.5,1524.5,4856.520256,53.850608,-27.056279,53.850641,-27.056273,53.850676,53.850676,-27.056260,-27.056260,53.850653,-27.056261,53.850614,-27.056299,53.850641,-27.056273,53.850618,-27.056275,53.850641,-27.056273,53.850608,-27.056279,53.850641,-27.056273,-0.001477,0.02

In [24]:
df['uberID']=df.index

In [25]:
df = df.rename(columns={'cluster_label':'cluster_label_u'})

In [26]:
# Saving
df.to_csv('../../data/WD03/WD03_UMAP_full.csv')

In [27]:
columns_to_save = ['uberID', 'RAmax', 'Decmax', 'mag_Zt','final_label', 'missing_bands', 'umap1', 'umap2', 'umap3', 'umap4', 'umap5', 'umap6', 'umap7', 'umap8', 'umap9', 'umap10', 'class',
                   'cluster_label_u', 'cluster_label_no_u', 'label_type', 'umap1_u', 'umap2_u', 'umap3_u', 'umap4_u', 'umap5_u', 'umap6_u', 'umap7_u', 'umap8_u', 'umap9_u', 'umap10_u',
                   'umap1_no_u', 'umap2_no_u', 'umap3_no_u', 'umap4_no_u', 'umap5_no_u', 'umap6_no_u', 'umap7_no_u', 'umap8_no_u', 'umap9_no_u', 'umap10_no_u']

In [28]:
df[columns_to_save].to_csv('../../data/WD03/WD03_UMAP_reduced_columns.csv')